In [191]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits

In [192]:
z_bin_centre = 2.0
z_bin_width = 0.2
z_bin = (z_bin_centre-z_bin_width/2,z_bin_centre+z_bin_width/2)
beta = 1.0

In [202]:
#Get the gaussian delta skewers
h = fits.open('/Users/jfarr/Projects/repixelise/test_output/test_multi/0/10/picca-gaussian-8-10.fits')
delta_G_rows = h[1].data.T
#delta_G_rows = delta_G_rows[0,:]

#Trim the skewers to a narrow window around the z value selected
#Get the growth factor and any other useful things
master = fits.open('/Users/jfarr/Projects/repixelise/test_output/test_multi/nside_8_master.fits')
D = master[2].data['D'][-571:]
Z = master[2].data['Z'][-571:]
valid_start = np.argmax(Z>z_bin[0])
valid_end = np.argmax(Z>z_bin[1]) - 1

D = D[valid_start:valid_end]
Z = Z[valid_start:valid_end]
delta_G_rows = delta_G_rows[:,valid_start:valid_end]

print('we have {} skewers, each with {} cells'.format(delta_G_rows.shape[0],delta_G_rows.shape[1]))
print('actual z-range of cells: {:2.2f} < z < {:2.2f}'.format(np.min(Z),np.max(Z)))

we have 6579 skewers, each with 74 cells
actual z-range of cells: 1.92 < z < 2.10


In [241]:
def lognormal_transform(delta_G_rows,sigma_G,D):
    density_rows = np.zeros(delta_G_rows.shape)
    for j in range(delta_G_rows.shape[1]):
        D_j = D[j]
        density_rows[:,j] = np.exp(D_j*delta_G_rows[:,j]-(D_j**2)*(sigma_G**2)/2.)
    return density_rows

In [298]:
def get_mean_F_model(z):
    return np.exp((np.log(0.8))*((1+z)/3.25)**3.2)

In [299]:
def get_flux_stats(z,sigma_G,alpha,beta,delta_G_rows):
        
    density_rows = lognormal_transform(delta_G_rows,sigma_G,D)

    delta_G_data = np.reshape(delta_G_rows,delta_G_rows.shape[0] * delta_G_rows.shape[1])
    density_data = np.reshape(density_rows,density_rows.shape[0] * density_rows.shape[1])
    
    F_data = np.exp(-alpha*(density_data**beta))
    
    mean_F = np.average(F_data)

    delta_F_data = F_data/mean_F - 1

    prob_delta_G = (1/(np.sqrt(2*np.pi)*sigma_G))*np.exp(-(delta_G_data**2)/(2*sigma_G**2))
    
    sigma_F = np.sqrt(np.trapz(prob_delta_G*delta_F_data,delta_G_data))
    
    return mean_F, sigma_F

In [300]:
def search_parameter_monotonic(start_parameter,tolerance):
    return

In [318]:
sigma_G_values = np.linspace(2,10,20)
print('{} <= sigma_G <= {}'.format(np.min(sigma_G_grid),np.max(sigma_G_grid)))
print('{} values of sigma_G will be tested'.format(sigma_G_grid.shape[0]))

2.0 <= sigma_G <= 10.0
20 values of sigma_G will be tested


In [319]:
tolerance = 0.0001

mean_F_model = get_mean_F_model(z)
print('mean F model {}\n'.format(mean_F_model))

for sigma_G in sigma_G_values:
    
    alpha_low = 0.0
    alpha_high = 1000.0
    
    count = 0
    exit = 0
    
    while exit == 0 and count < 100:
        alpha_midpoint = (alpha_low + alpha_high)/2.0

        mean_F_al,sigma_F_al = get_flux_stats(z,sigma_G,alpha_low,beta,delta_G_rows)
        mean_F_am,sigma_F_am = get_flux_stats(z,sigma_G,alpha_midpoint,beta,delta_G_rows)
        mean_F_ah,sigma_F_ah = get_flux_stats(z,sigma_G,alpha_high,beta,delta_G_rows)

        #print('model: {:2.2f}, ({:2.2f},{:2.2f},{:2.2f}), ({:2.2f},{:2.2f},{:2.2f}), ({:2.2f},{:2.2f},{:2.2f})'.format(mean_F_model,alpha_low,alpha_midpoint,alpha_high,mean_F_al,mean_F_am,mean_F_ah,sigma_F_al,sigma_F_am,sigma_F_ah))
        if np.sign(mean_F_al-mean_F_model) * np.sign(mean_F_am-mean_F_model) > 0:
            alpha_low = alpha_midpoint
        else:
            alpha_high = alpha_midpoint
                
        if abs(mean_F_am/mean_F_model - 1) < tolerance:
            exit = 1
        else:
            count += 1
            
        alpha = alpha_midpoint
        mean_F = mean_F_am
        sigma_F = sigma_F_am
    
    print('sigma G {:2.2f}, alpha {:2.2f}: mean F {:2.4f}, sigma F {:2.4f}, n {:3d}'.format(sigma_G,alpha,mean_F,sigma_F,count))

mean F model 0.8413721721350615



/Users/jfarr/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in sqrt
  app.launch_new_instance()


sigma G 2.00, alpha 0.17: mean F 0.8413, sigma F nan, n  20
sigma G 2.42, alpha 0.20: mean F 0.8414, sigma F nan, n  21
sigma G 2.84, alpha 0.24: mean F 0.8414, sigma F nan, n  20
sigma G 3.26, alpha 0.30: mean F 0.8413, sigma F 0.1001, n  17
sigma G 3.68, alpha 0.39: mean F 0.8414, sigma F 0.2186, n  19
sigma G 4.11, alpha 0.52: mean F 0.8413, sigma F 0.2827, n  19
sigma G 4.53, alpha 0.72: mean F 0.8414, sigma F 0.3272, n  18
sigma G 4.95, alpha 1.02: mean F 0.8414, sigma F 0.3600, n  15
sigma G 5.37, alpha 1.50: mean F 0.8414, sigma F 0.3845, n  18
sigma G 5.79, alpha 2.27: mean F 0.8414, sigma F 0.4021, n  15
sigma G 6.21, alpha 3.55: mean F 0.8414, sigma F 0.4139, n  17
sigma G 6.63, alpha 5.72: mean F 0.8414, sigma F 0.4201, n  15
sigma G 7.05, alpha 9.51: mean F 0.8414, sigma F 0.4211, n  16
sigma G 7.47, alpha 16.31: mean F 0.8414, sigma F 0.4166, n  15
sigma G 7.89, alpha 28.84: mean F 0.8414, sigma F 0.4063, n  14
sigma G 8.32, alpha 52.61: mean F 0.8414, sigma F 0.3897, n  1